In [11]:
from word2vec_functions import *
import h2o
h2o.init()
import numpy as np
import pandas as pd
from math import sqrt
from h2o.estimators.word2vec import H2OWord2vecEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator
import sklearn
from sklearn import ensemble
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer


Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,12 mins 19 secs
H2O_cluster_timezone:,America/Chicago
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.1
H2O_cluster_version_age:,"28 days, 17 hours and 48 minutes"
H2O_cluster_name:,H2O_from_python_csridlen_366ufw
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.321 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [2]:
# Initializing word2vec model
airbnb_path = "../data/newnh_airbnb_2021.csv"
airbnb_names = h2o.import_file(airbnb_path, destination_frame = "airbnbnames",
                             header = 1)
# Create word vectors
words = tokenize(airbnb_names["name"])
w2v_model = H2OWord2vecEstimator(sent_sample_rate = 0.0, epochs = 10)
w2v_model.train(training_frame = words)
airbnb_names_vecs = w2v_model.transform(words, aggregate_method = "AVERAGE")
valid_airbnb_names = ~ airbnb_names_vecs["C1"].isna()
data = airbnb_names[valid_airbnb_names,:].cbind(airbnb_names_vecs[valid_airbnb_names,:])
data_split = data.split_frame(ratios=[0.8])

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
word2vec Model Build progress: |█████████████████████████████████████████████████| (done) 100%


In [3]:
gbm_model = H2OGradientBoostingEstimator()
gbm_model.train(x = airbnb_names_vecs.names,
                y= "price", 
                training_frame = data_split[0], 
                validation_frame = data_split[1])

# We can't just rely on the names because RMSE is pretty high

varimp = gbm_model.varimp(use_pandas=True)
top_20 = list(varimp.head(20)["variable"])
print(top_20)

gbm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
['C16', 'C71', 'C24', 'C58', 'C69', 'C56', 'C64', 'C98', 'C38', 'C89', 'C99', 'C72', 'C10', 'C43', 'C30', 'C63', 'C96', 'C41', 'C74', 'C14']


In [4]:
## In word2vec.ipynb, we see the top 20 most important categorizations
# Let's filter just for those 20
top_20_list = list(set(airbnb_names_vecs.names)  & set(top_20))
airbnb_names_vecs = airbnb_names_vecs[valid_airbnb_names, :]
airbnb_names_vecs = airbnb_names_vecs[top_20_list]
airbnb_names_vecs
#data = airbnb_names[valid_airbnb_names,:].cbind(airbnb_names_vecs[valid_airbnb_names,:])
#data

C89,C58,C24,C16,C98,C63,C43,C69,C14,C64,C41,C99,C56,C96,C72,C10,C38,C74,C30,C71
0.0414929,-0.314065,-0.170191,0.314494,-0.0019736,0.0560044,-0.143456,-0.0355399,0.0873906,-0.163774,-0.0800597,0.042211,-0.0472068,-0.204014,-0.0258083,0.142096,0.171781,0.013836,0.0646204,0.00276503
-0.0488967,-0.197986,-0.240912,0.135397,-0.0521394,0.118144,-0.055534,0.108907,-0.0797843,-0.0179494,-0.0872375,-0.227657,0.018124,-0.0953801,-0.0809367,-0.048385,-0.0496451,0.0436898,-0.0267796,-0.105073
0.0388408,-0.159724,-0.31335,0.198705,0.0109361,0.0219084,0.0164988,-0.0348356,0.206743,0.0197257,-0.116792,-0.0200244,0.0864699,-0.142611,-0.0994022,-0.130285,-0.0193499,0.00216912,0.185774,0.1622
-0.197551,-0.118933,-0.00532131,0.190949,0.0755526,-0.174513,0.145527,-0.0351942,-0.149814,0.047343,0.0177735,-0.0793491,0.217482,-0.338033,0.0609829,0.0924168,0.13096,0.0288448,-0.232615,-0.318538
-0.0347637,-0.221408,0.0432626,0.188089,-0.210913,0.0625676,0.0890583,-0.19489,-0.257446,0.139886,-0.128479,-0.0790875,0.206579,-0.124553,-0.00791913,-0.114589,0.142946,0.0640596,0.160911,-0.213308
0.0202626,-0.179782,-0.110493,0.222263,0.00602872,-0.00599562,-0.0946381,-0.0366089,-0.00950434,-0.0810127,-0.141905,-0.0109641,0.255121,-0.0806258,0.0463872,0.101333,0.140894,0.0047433,0.0783908,-0.213151
-0.200809,-0.146775,-0.122234,-0.0616028,0.119583,0.0666368,-0.0348425,-0.182572,-0.226643,-0.0519168,0.226666,-0.0902048,0.0371262,-0.257692,-0.0456282,0.290187,0.07071,0.150159,-0.0514339,-0.213527
-0.0205146,-0.258371,-0.181212,0.257406,-0.124268,-0.0945426,0.0264063,-0.0326816,-0.169061,0.131563,-0.163537,-0.0920382,0.0527756,-0.15814,-0.0473768,0.118396,-0.0181307,0.0759548,0.103577,-0.0419811
-0.306076,-0.0615464,0.10615,0.283102,0.0796312,-0.104361,0.151458,-0.0417312,0.188649,0.0812921,0.0623345,-0.109441,0.123716,-0.110972,-0.0726835,0.134639,-0.135533,0.0357085,0.122675,-0.113716
-0.00762284,-0.0329112,-0.00158766,0.118969,-0.150835,-0.151157,-0.00870421,-0.0389306,-0.0340204,-0.0290756,-0.039602,0.00227246,0.0687493,-0.064668,-0.0459878,-0.0338405,-0.0409418,0.0460234,0.0843163,-0.273347


In [5]:
data = airbnb_names[valid_airbnb_names,:].cbind(airbnb_names_vecs).as_data_frame()
data.dtypes

id                                  int64
name                               object
host_id                             int64
host_name                          object
neighbourhood_group                object
neighbourhood                      object
latitude                          float64
longitude                         float64
room_type                          object
price                               int64
minimum_nights                      int64
number_of_reviews                   int64
last_review                       float64
reviews_per_month                 float64
calculated_host_listings_count      int64
availability_365                    int64
number_of_reviews_ltm               int64
license                           float64
new_neighbourhood                  object
C89                               float64
C58                               float64
C24                               float64
C16                               float64
C98                               

In [6]:
not_used = ["id", "host_id", "name", "host_name", "neighbourhood", "latitude", "longitude", "price", "last_review"]
X = data.drop(not_used, axis = 1)
# We want numbers to be of same type
dict_ints = {'minimum_nights': float,
             'number_of_reviews': float,
             'calculated_host_listings_count': float,
             'availability_365': float,
             'number_of_reviews_ltm': float
}
X = X.astype(dict_ints)
dict_cats = { 
    "neighbourhood_group": "category",
    "room_type": "category",
    "new_neighbourhood": "category"
}
list_cats = list(dict_cats.keys())

X = X.astype(dict_cats)
X.dtypes


# Convert y to float as well
y = data["price"].astype(float)

# One hot encode room type
encoder = OneHotEncoder(handle_unknown = "ignore")
encoder_df = pd.DataFrame(encoder.fit_transform(X[["room_type"]]).toarray())
# Merge with original dataset
X_df = X.join(encoder_df)

# Filter for top neighborhoods

# One hot encode neighborhoods
encoder_n = pd.DataFrame(encoder.fit_transform(X[["neighbourhood_group"]]).toarray())
encoder_n.columns = [4, 5, 6, 7, 8]
# Merge with original dataset
X = X_df.join(encoder_n)
# Drop original categorical vartiables
X.dtypes
X = X.drop(list_cats, axis = 1)
X.dtypes # all variables are floats
X = X.fillna(0)


In [17]:
## Create gradient-boosted model
# All feature names must be strings
X.columns = X.columns.astype(str)
# Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
params = {
    "n_estimators": 500,
    "max_depth": 10,
    "min_samples_split": 5,
    "learning_rate": 0.01,
    "loss": "squared_error"}
reg = ensemble.GradientBoostingRegressor(**params)
reg.fit(X_train, y_train)

GradientBoostingRegressor(learning_rate=0.01, max_depth=10, min_samples_split=5,
                          n_estimators=500)

In [18]:
from sklearn.metrics import mean_squared_error
rmse = sqrt(mean_squared_error(y_test, reg.predict(X_test))) # manually extract rmse from mse
print(rmse) #out-of-sample rmse

230.14216221806637


In [ ]:
# variable importance


In [ ]:
names = pd.read_csv('../data/newnh_airbnb_2021.csv')[["id", "name"]] # use this for later